In [ ]:
import pandas as pd
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [7]:
# Load the Excel codebook
file_path = "data/codebooks/codebook_colleges_ipeds_directory.xlsx"

# Load the first sheet
df = pd.read_excel(file_path, sheet_name='variables')

df['source'] = 'ipeds'
df['endpoint_name'] = 'directory'

- Next here to scale across all codebooks

In [12]:
df.head()

,variable,format,label,source,endpoint_name
0,unitid,numeric,Unit ID number,ipeds,directory
1,year,numeric,Academic year (fall semester),ipeds,directory
2,opeid,string,8-digit Office of Postsecondary Education (OPE...,ipeds,directory
3,inst_name,string,Institution (entity) name,ipeds,directory
4,inst_alias,string,Institution name alias,ipeds,directory


In [ ]:

# Get the URI and authentication credentials from environment variables
URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))

# initalize the driver
driver = GraphDatabase.driver(URI, auth=AUTH)


In [20]:
# Attempt to connect to Neo4j Aura
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            result = session.run("RETURN 1 AS test")
            for record in result:
                print("Connected! Test Query Result:", record["test"])
except Exception as e:
    print("Error:", e)

Connected! Test Query Result: 1


In [ ]:
# Function to run a query
def run_query(query, parameters):
    with driver.session() as session:
        session.run(query, parameters)

# Create Nodes and Relationships in Neo4j
def create_knowledge_graph(data):
    for _, row in data.iterrows():
        variable_name = row['variable']
        format = row['format']
        label = row['label']
        source = row['source']
        endpoint_name = row['endpoint_name']

        # Create Variable node
        variable_query = """
        MERGE (v:Variable {name: $name})
        ON CREATE SET v.format = $format, v.label = $label
        """
        run_query(variable_query, {"name": variable_name, "format": format, "label": label})

        # Create Source node
        source_query = """
        MERGE (s:Source {name: $name})
        """
        run_query(source_query, {"name": source})

        # Create Endpoint Name node
        endpoint_query = """
        MERGE (e:Endpoint {name: $name})
        """
        run_query(endpoint_query, {"name": endpoint_name})

        # Create Relationships
        relationship_query_1 = """
        MATCH (v:Variable {name: $variable_name})
        MATCH (e:Endpoint {name: $endpoint_name})
        MERGE (v)-[:EXISTS_IN]->(e)
        """
        run_query(relationship_query_1, {"variable_name": variable_name, "endpoint_name": endpoint_name})

        relationship_query_2 = """
        MATCH (e:Endpoint {name: $endpoint_name})
        MATCH (s:Source {name: $source})
        MERGE (e)-[:PROVIDED_BY]->(s)
        """
        run_query(relationship_query_2, {"endpoint_name": endpoint_name, "source": source})

        relationship_query_3 = """
        MATCH (v:Variable {name: $variable_name})
        MATCH (s:Source {name: $source})
        MERGE (v)-[:PROVIDED_BY]->(s)
        """
        run_query(relationship_query_3, {"variable_name": variable_name, "source": source})

# Call the function with your DataFrame
create_knowledge_graph(df)

print("Knowledge graph populated!")

/var/folders/cc/rk1x9by94ks5m201ycvs1b0m0000gn/T/ipykernel_50853/3368087144.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Knowledge graph populated!


In [22]:
# Check the data in the graph
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' AS message")
        for record in result:
            print(record["message"])

Connection successful!


# Query the Graph

In [ ]:
def get_variables_and_sources():
    query = """
    MATCH (v:Variable)-[:EXISTS_IN]->(e:Endpoint)-[:PROVIDED_BY]->(s:Source)
    RETURN v.name AS variable, v.label AS label, e.name AS endpoint, s.name AS source
    """
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            print(
                f"Variable: {record['variable']}, "
                f"Label: {record['label']}, "
                f"Endpoint: {record['endpoint']}, "
                f"Source: {record['source']}"
            )

# Call the function to test
get_variables_and_sources()

In [ ]:
def get_relationships():
    query = """
    MATCH (a)-[r]->(b)
    RETURN a.name AS from_node, type(r) AS relationship, b.name AS to_node
    """
    with driver.session() as session:
        result = session.run(query)
        print("Relationships in the Graph:")
        for record in result:
            print(
                f"From: {record['from_node']}, "
                f"Relationship: {record['relationship']}, "
                f"To: {record['to_node']}"
            )

# Call the function to test
get_relationships()